<a href="https://colab.research.google.com/github/SurrajKumar2000/Lung-and-Colon-Cancer-prediction-using-CNN/blob/main/M2_XCEPTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## XCEPTION NET ARCHITECTURE

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow
import tensorflow as tf
print(tf.__version__)

import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

2.12.0


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split = 0.2)
train_data=train.flow_from_directory(directory = 'DATASET/TRAIN',target_size=(224,224),
                                     batch_size=32,class_mode='categorical')

Found 2000 images belonging to 5 classes.


In [ ]:
test=ImageDataGenerator(rescale=1./255)
test_data=test.flow_from_directory(directory = 'DATASET/TEST',target_size=(224,224),
                                   batch_size=32,class_mode='categorical')

Found 2000 images belonging to 5 classes.


In [ ]:
def xception(input_shape, n_classes):

  def conv_bn(x, f, k, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p, use_bias=False)(x)
    x = BatchNormalization()(x)
    return x


  def sep_bn(x, f, k, s=1, p='same'):
    x = SeparableConv2D(f, k, strides=s, padding=p, use_bias=False)(x)
    x = BatchNormalization()(x)
    return x


  def entry_flow(x):
    x = conv_bn(x, 32, 3, 2)
    x = ReLU()(x)
    x = conv_bn(x, 64, 3)
    tensor = ReLU()(x)

    x = sep_bn(tensor, 128, 3)
    x = ReLU()(x)
    x = sep_bn(x, 128, 3)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, 128, 1, 2)

    x = Add()([tensor, x])
    x = ReLU()(x)
    x = sep_bn(x, 256, 3)
    x = ReLU()(x)
    x = sep_bn(x, 256, 3)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, 256, 1, 2)

    x = Add()([tensor, x])
    x = ReLU()(x)
    x = sep_bn(x, 728, 3)
    x = ReLU()(x)
    x = sep_bn(x, 728, 3)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, 728, 1, 2)
    x = Add()([tensor, x])

    return x


  def middle_flow(tensor):
    for _ in range(8):
      x = ReLU()(tensor)
      x = sep_bn(x, 728, 3)
      x = ReLU()(x)
      x = sep_bn(x, 728, 3)
      x = ReLU()(x)
      x = sep_bn(x, 728, 3)

      tensor = Add()([tensor, x])

    return tensor


  def exit_flow(tensor):
    x = ReLU()(tensor)
    x = sep_bn(x, 728, 3)
    x = ReLU()(x)
    x = sep_bn(x, 1024, 3)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, 1024, 1, 2)

    x = Add()([tensor, x])
    x = sep_bn(x, 1536, 3)
    x = ReLU()(x)
    x = sep_bn(x, 2048, 3)
    x = ReLU()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(n_classes, activation='softmax')(x)

    return x


  input = Input(input_shape)

  x = entry_flow(input)
  x = middle_flow(x)
  output = exit_flow(x)

  model = Model(input, output)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',tensorflow.keras.metrics.Precision()])
  return model


input_shape = 224, 224, 3
n_classes = 5

K.clear_session()
model = xception(input_shape, n_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

 re_lu_6 (ReLU)                 (None, 28, 28, 728)  0           ['batch_normalization_8[0][0]']  
                                                                                                  
 separable_conv2d_5 (SeparableC  (None, 28, 28, 728)  536536     ['re_lu_6[0][0]']                
 onv2D)                                                                                           
                                                                                                  
 conv2d_4 (Conv2D)              (None, 14, 14, 728)  186368      ['batch_normalization_7[0][0]']  
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 28, 28, 728)  2912       ['separable_conv2d_5[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 batch_nor

 Conv2D)                                                                                          
                                                                                                  
 batch_normalization_18 (BatchN  (None, 14, 14, 728)  2912       ['separable_conv2d_13[0][0]']    
 ormalization)                                                                                    
                                                                                                  
 re_lu_15 (ReLU)                (None, 14, 14, 728)  0           ['batch_normalization_18[0][0]'] 
                                                                                                  
 separable_conv2d_14 (Separable  (None, 14, 14, 728)  536536     ['re_lu_15[0][0]']               
 Conv2D)                                                                                          
                                                                                                  
 batch_nor

 batch_normalization_27 (BatchN  (None, 14, 14, 728)  2912       ['separable_conv2d_22[0][0]']    
 ormalization)                                                                                    
                                                                                                  
 re_lu_24 (ReLU)                (None, 14, 14, 728)  0           ['batch_normalization_27[0][0]'] 
                                                                                                  
 separable_conv2d_23 (Separable  (None, 14, 14, 728)  536536     ['re_lu_24[0][0]']               
 Conv2D)                                                                                          
                                                                                                  
 batch_normalization_28 (BatchN  (None, 14, 14, 728)  2912       ['separable_conv2d_23[0][0]']    
 ormalization)                                                                                    
          

 batch_normalization_36 (BatchN  (None, 14, 14, 1024  4096       ['separable_conv2d_31[0][0]']    
 ormalization)                  )                                                                 
                                                                                                  
 batch_normalization_37 (BatchN  (None, 7, 7, 1024)  4096        ['conv2d_5[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 7, 7, 1024)  0           ['batch_normalization_36[0][0]'] 
                                                                                                  
 add_11 (Add)                   (None, 7, 7, 1024)   0           ['batch_normalization_37[0][0]', 
                                                                  'max_pooling2d_3[0][0]']        
          